In [ ]:
# load the library for path
from pathlib import Path
import pandas as pd
import openpyxl as xl
import pandas as pd
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
from statsmodels.tsa.stattools import adfuller
from pmdarima import auto_arima
from sklearn.metrics import mean_squared_error, mean_absolute_error
from numpy import *

In [ ]:
# import data set
data_dir = Path('../datathon')

# read xlsx data set
df = pd.read_csv(data_dir/'VacancyRate.csv', index_col = [0], parse_dates=[0])

# remove all the empty column
df = df.dropna(axis=1, how='all')
df.dropna(inplace=True)
df.index.format = '%Y-%m-%d'

plt.figure(figsize=(10, 6))
plt.plot(df)
plt.legend(loc = 'best')
plt.title('Vacancy Rate')
plt.show()

# 3. Test for Data Set
## 3.1 Stationary Test

In [ ]:
for i in range(df.shape[1]):
    column_name = df.columns[i]
    column_data = df.iloc[:, i]  # Access column data using iloc

    # Perform the ADF test
    adft = adfuller(column_data, autolag='AIC')
    
    if df.iloc[:, i].isna().any():
        df = df.dropna()

    # Create a DataFrame to display the ADF test results
    output_df = pd.DataFrame({
        "Values": [adft[0], adft[1], adft[2], adft[3], adft[4]['1%'], adft[4]['5%'], adft[4]['10%']],
        "Metric": ["Test Statistics", "p-value", "No. of lags used", "Number of observations used",
                   "critical value (1%)", "critical value (5%)", "critical value (10%)"]
    })

    # Print the column name and ADF test results
    print(f"Results for {column_name}:")
    print(output_df, "\n")


## 3.2 Autocorrelation

In [ ]:
for i in range(df.shape[1]):
    column_name = df.columns[i]
    print(f"Results for {column_name}:")

    autocorrelation_lag1 = df.iloc[:, i].autocorr(lag=1)
    print("One Month Lag: ", autocorrelation_lag1)

    autocorrelation_lag3 = df.iloc[:, i].autocorr(lag=3)
    print("Three Month Lag: ", autocorrelation_lag3)

    autocorrelation_lag6 = df.iloc[:, i].autocorr(lag=6)
    print("Six Month Lag: ", autocorrelation_lag6)

    autocorrelation_lag9 = df.iloc[:, i].autocorr(lag=9)
    print("Nine Month Lag: ", autocorrelation_lag9)
    
    print("\n")

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

for i in range(df.shape[1]):
    column_name = df.columns[i]
    fig, ax = plt.subplots(2, figsize=(12,6))
    ax[0] = plot_acf(df.iloc[:, i], ax=ax[0], lags=6, title=f"Autocorrelation for {column_name}")
    ax[1] = plot_pacf(df.iloc[:, i], ax=ax[1], lags=6, title=f"Partial Autocorrelation for {column_name}")

## 3.3 Decomposition

# 4. Data Transformation

In [ ]:
df_diff = df.diff().dropna()

for i in range(df.shape[1]):
    column_name = df.columns[i]
    print(f"Results for {column_name}:")
    print("After differencing:")

    # perform the stationarity test
    adft = adfuller(df_diff.iloc[:, i], autolag='AIC')
    output_df = pd.DataFrame({"Values":[adft[0],adft[1],adft[2],adft[3], adft[4]['1%'], adft[4]['5%'], adft[4]['10%']]  , "Metric":["Test Statistics","p-value","No. of lags used","Number of observations used", 
                                                        "critical value (1%)", "critical value (5%)", "critical value (10%)"]})
    print(output_df)
    print("\n")

    # autocorrelation
    from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

    fig, ax = plt.subplots(2, figsize=(12,6))
    ax[0] = plot_acf(df_diff.iloc[:,i], ax=ax[0], lags=6, title= f"Autocorrelation for {column_name}")
    ax[1] = plot_pacf(df_diff.iloc[:,i], ax=ax[1], lags=6, title= f"Partial Autocorrelation for {column_name}")
    

In [ ]:
# convert df_diff to data frame
df_diff = pd.DataFrame(df_diff)
df_diff.columns

In [ ]:
# second order differencing
df_diff_diff = df_diff['Pulau Pinang'].diff().dropna()
df_diff_diff = df_diff['W.P Kuala Lumpur'].diff().dropna()
df_diff_diff = df_diff['Pulau Pinang'].diff().dropna()

# 5. Fitting the Model

# Split the Data Set 

In [ ]:
train = df.iloc[:round(len(df)*0.8)]
test = df.iloc[round(len(df)*0.8):]
train.shape, test.shape

## ETS

In [ ]:
def find_ETS(train,test,column_number):
    Trend_choice = ['add','mul']
    Seasonal_choice = ['add','mul']

    MSE = pd.DataFrame({'Trend': [], 'Seasonal': [], 'MSE': [], 'RMSE': [], 'MAE': []})

    for value in Trend_choice:
        for value2 in Seasonal_choice:
            model = ExponentialSmoothing(train.iloc[:, column_number], trend=value, seasonal=value2, seasonal_periods=6)  
            model_fit = model.fit()
            forecasts = model_fit.forecast(steps=3)
            # Check the error measure for each model and update the MSE_johor DataFrame
            new_row = pd.DataFrame({'Trend': value, 
                                    'Seasonal': value2, 
                                    'MSE': mean_squared_error(test.iloc[:, column_number], forecasts),
                                    'RMSE': np.sqrt(mean_squared_error(test.iloc[:, column_number], forecasts)),
                                    'MAE': mean_absolute_error(test.iloc[:, column_number], forecasts)}, index=[0]) 
            MSE = pd.concat([MSE, new_row], ignore_index=True)
    return MSE


In [ ]:
ETS_johor = find_ETS(train,test,0)
ETS_kedah = find_ETS(train,test,1)
ETS_kelantan = find_ETS(train,test,2)
ETS_melaka = find_ETS(train,test,3)
ETS_n9 = find_ETS(train,test,4)
ETS_pahang = find_ETS(train,test,5)
ETS_penang = find_ETS(train,test,6)
ETS_perak = find_ETS(train,test,7)
ETS_perlis = find_ETS(train,test,8)
ETS_selangor = find_ETS(train,test,9)
ETS_terengganu = find_ETS(train,test,10)
ETS_sabah = find_ETS(train,test,11)
ETS_sarawak = find_ETS(train,test,12)
ETS_wpkl = find_ETS(train,test,13)
ETS_wplabuan = find_ETS(train,test,14)
ETS_p = find_ETS(train,test,15)
ETS_Total = find_ETS(train,test,16)

# sort the output above by MSE ascending
ETS_johor.sort_values(by=['MSE'])
ETS_kedah.sort_values(by=['MSE'])
ETS_kelantan.sort_values(by=['MSE'])
ETS_melaka.sort_values(by=['MSE'])
ETS_n9.sort_values(by=['MSE'])
ETS_pahang.sort_values(by=['MSE'])
ETS_penang.sort_values(by=['MSE'])
ETS_perak.sort_values(by=['MSE'])
ETS_perlis.sort_values(by=['MSE'])
ETS_selangor.sort_values(by=['MSE'])
ETS_terengganu.sort_values(by=['MSE'])
ETS_sabah.sort_values(by=['MSE'])
ETS_sarawak.sort_values(by=['MSE'])
ETS_wpkl.sort_values(by=['MSE'])
ETS_wplabuan.sort_values(by=['MSE'])
ETS_p.sort_values(by=['MSE'])
ETS_Total.sort_values(by=['MSE'])

# combine first row of output above into one dataframe
ETS = pd.concat([ETS_johor.iloc[0], ETS_kedah.iloc[0], ETS_kelantan.iloc[0], ETS_melaka.iloc[0], ETS_n9.iloc[0], ETS_pahang.iloc[0], ETS_penang.iloc[0], ETS_perak.iloc[0], ETS_perlis.iloc[0], ETS_selangor.iloc[0], ETS_terengganu.iloc[0], ETS_sabah.iloc[0], ETS_sarawak.iloc[0], ETS_wpkl.iloc[0], ETS_wplabuan.iloc[0], ETS_p.iloc[0],ETS_Total.iloc[0]], axis=1)
ETS = ETS.T
ETS.index = train.columns

## ARIMA test

In [ ]:
ARIMA_forecast = []
ARIMA_forecast = pd.DataFrame(ARIMA_forecast)

for i in range(df.shape[1]):
    column_name = df.columns[i]
    print(f"Results for {column_name}:")

    ARIMAmodel = auto_arima(train[[column_name]], trace=True, error_action='ignore', suppress_warnings=True)
    ARIMAmodel.fit(train[[column_name]])
    forecast = ARIMAmodel.predict(n_periods=len(test))
    ARIMA_forecast = pd.concat([ARIMA_forecast, forecast], axis=1)

In [ ]:
test

In [ ]:
ARIMA_forecast.index = ['2022-01-07', '2022-01-10', '2023-01-01']
ARIMA_forecast.index = pd.to_datetime(ARIMA_forecast.index)

In [ ]:
for i in range(df.shape[1]):
    column_name = df.columns[i]
    column_data = df.iloc[:, i]  # Access column data using iloc
    
    # plot the predictions for validation set
    # set the starting date at 2019-01-01
    plt.figure(figsize=(12,8))
    plt.plot(train.iloc[:,i], label='Train')
    plt.plot(test.iloc[:,i], label='test data')
    plt.plot(ARIMA_forecast.iloc[:,i], label='Prediction')
    plt.title(f'ARIMA Forecast for {column_name}')
    plt.legend(loc='best')

### ARIMA Accuracy Testing

In [ ]:
# make a empty data frame to store the error
error_df_ARIMA = pd.DataFrame({'MSE': [], 'RMSE': [], 'MAE': []})

for i in range(df.shape[1]):
    mae = mean_absolute_error(test.iloc[:,i], ARIMA_forecast.iloc[:,i])
    mse = mean_squared_error(test.iloc[:,i], ARIMA_forecast.iloc[:,i])
    rmse = np.sqrt(mse)
    # Create a DataFrame from the current values
    new_row = pd.DataFrame({'MSE': [mse], 'RMSE': [rmse], 'MAE': [mae]})

    # Concatenate the new DataFrame with the existing error_df_ARIMA
    error_df_ARIMA = pd.concat([error_df_ARIMA, new_row], ignore_index=True)

error_df_ARIMA.index = test.columns


In [ ]:
# compare all the error from error_df_ETS and error_df_ARIMA by row 
ETS['Model'] = 'ETS'
error_df_ARIMA['Model'] = 'ARIMA'
print(ETS)
print(error_df_ARIMA)

In [ ]:
# Johor - ARIMA
# Kedah - ETS
# Kelantan - ARIMA
# Melaka - ETS
# N9 - ETS
# Pahang - ETS
# Pulau Pinang - ARIMA
# Perak - ARIMA
# Perlis - ETS
# Selangor - ARIMA
# Terengganu - ETS
# Sabah - ETS
# Sarawak - ETS
# Kuala Lumpur - ARIMA
# Labuan - ARIMA
# Putrajaya - ETS
# Total - ARIMA